We'll go back to our Julia example for the lab. We re-produce our last step here...

In [1]:
from collections import namedtuple
Box = namedtuple("Box", "x1 x2 y1 y2")
bounds = Box(-1.8, 1.8, -1.8, 1.8)
focus=complex(-0.62772, -.42193)
gridsize=1000
iters=300

def setup_grid(gridsize, box):
    xstep = (box.x2 - box.x1)/(gridsize - 1.0)
    ystep = (box.y2 - box.y1)/(gridsize - 1.0)
    xs = (box.x1+ i* xstep for i in range(gridsize))
    zs=[]
    for x in xs:
        ys = (box.y1+ i* ystep for i in range(gridsize))
        for y in ys:
            zs.append(complex(x,y))
    return zs

def zts1(maxiter, zs, c): 
    output = [0] * len(zs)
    for i,z in enumerate(zs):
        n=0
        while n < maxiter and abs(z) < 2:
            z=z*z+c
            n+=1 
        output[i] = n
    return output

def run1():
    zs = setup_grid(gridsize, bounds)
    out = zts1(iters, zs, focus)
    return zs, out

If you remember, profiling found that `zts1` needed speeding up...

In [2]:
%timeit -r 1 -n 5 run1()

5 loops, best of 1: 11.1 s per loop


### Q1

What speedup do you get from simply cythonizing `zts1`, with no annotations at all?

In [4]:

%load_ext Cython

In [6]:
%%cython --annotate
def zts1_cython(maxiter, zs, c): 
    #your code here
    output = [0] * len(zs)
    for i,z in enumerate(zs):
        n=0
        while n < maxiter and abs(z) < 2:
            z=z*z+c
            n+=1 
        output[i] = n
    return output


In [7]:
def run1():
    zs = setup_grid(gridsize, bounds)
    out = zts1_cython(iters, zs, focus)
    return zs, out

In [8]:
%timeit -r 1 -n 5 run1()

5 loops, best of 1: 6.19 s per loop


### Q2

Keeping `zts1` a `def` function, and leaving out `output` and `zs`, type annotate as many variables as you can. For complex numbers, use the type `double complex`. Create the function `zts1_cython2`

In [11]:
%%cython --annotate
#your code here
def zts1_cython2(int maxiter, zs, double complex c):
    #your code here
    cdef int i
    cdef double complex z
    cdef int n
    output = [0] * len(zs)
    for i, z in enumerate(zs):
        n=0
        while n < maxiter and abs(z) < 2:
            z=z*z+c
            n+=1 
        output[i] = n
    return output

In [12]:
def run2():
    zs = setup_grid(gridsize, bounds)
    out = zts1_cython2(iters, zs, focus)
    return zs, out

In [13]:
%timeit -r 1 -n 5 run2()

5 loops, best of 1: 3.65 s per loop


### Q3.

Replace the `abs` function in the `while` by an equivalent condition which does not require a square root (needed for `abs`. 

This equivalent but specialized code is called a **strength reduction**. You have lost flexibility and meaning in readability for faster speed.

In [21]:
%%cython --annotate
#your code here
def zts1_cython3(int maxiter, zs, double complex c):
    #your code here
    cdef int i
    cdef double complex z
    cdef int n
    output = [0] * len(zs)
    for i, z in enumerate(zs):
        n=0
        while n < maxiter and z.imag**2 + z.real**2 < 4:
            z=z*z+c
            n+=1 
        output[i] = n
    return output

In [22]:
def run3():
    zs = setup_grid(gridsize, bounds)
    out = zts1_cython3(iters, zs, focus)
    return zs, out

In [32]:
%timeit -r 1 -n 5 run3()

5 loops, best of 1: 695 ms per loop


### Q4.

Turn off bounds-checking (see cython docs for how). Does it make much of a difference? Why?

In [30]:
%%cython --annotate
cimport cython

@cython.boundscheck(False)
#your code here
def zts1_cython4(int maxiter, zs, double complex c):
    #your code here
    cdef int i
    cdef double complex z
    cdef int n
    output = [0] * len(zs)
    for i, z in enumerate(zs):
        n=0
        while n < maxiter and z.imag**2 + z.real**2 < 4:
            z=z*z+c
            n+=1 
        output[i] = n
    return output

In [33]:
def run4():
    zs = setup_grid(gridsize, bounds)
    out = zts1_cython4(iters, zs, focus)
    return zs, out

%timeit -r 1 -n 5 run4()

5 loops, best of 1: 695 ms per loop


### No further speedup because we have no C arrays in the function.  